In [1]:
from pyspark.sql import functions as F
from pyspark.sql.types import *
from pyspark.sql.window import Window

In [2]:
list=[['./.'],
      ['1/9'],
      ['4/8'],
      ['./.'],
      ['5/8']]



df=spark.createDataFrame(list, ['genotype_index'])

In [3]:
df.show()

+--------------+
genotype_index|
+--------------+
 ./.|
 1/9|
 4/8|
 ./.|
 5/8|
+--------------+

In [5]:
df1=df.withColumn('genotype_indices', F.split(F.col("genotype_index"), "/").cast("array<int>")).alias("genotype_indices")

+--------------+----------------+
genotype_index|genotype_indices|
+--------------+----------------+
 ./.| [0, 0]|
 1/9| [1, 9]|
 4/8| [4, 8]|
 ./.| [0, 0]|
 5/8| [5, 8]|
+--------------+----------------+

In [6]:
df.withColumn("genotype_indices", F.split(F.regexp_replace("genotype_index",'\.',""),'/').cast("array<int>")).show()

+--------------+----------------+
genotype_index|genotype_indices|
+--------------+----------------+
 ./.| [,]|
 1/9| [1, 9]|
 4/8| [4, 8]|
 ./.| [,]|
 5/8| [5, 8]|
+--------------+----------------+

In [7]:
from pyspark.sql import functions as F
from pyspark.sql.types import *

In [8]:
list=[['AA',['""'],['7']],
     ['AA',['9'],[]],
     ['AA',['""'],['2']],
     ['AA',[2],['""']]]
     


df=spark.createDataFrame(list, schema)

In [9]:

''''schema= StructType([StructField("Number", StringType()),
                   StructField("Matricule", ArrayType(StringType())),
                   StructField("Name", ArrayType(StringType()))])''''

File "<command-1146251761184632>" , line 4 
 StructField("Name", ArrayType(StringType()))])'''' 
 
^
 SyntaxError : EOL while scanning string literal

In [10]:
df.show()

+------+---------+----+
Number|Matricule|Name|
+------+---------+----+
 AA| [""]| [7]|
 AA| [9]| []|
 AA| [""]| [2]|
 AA| [2]|[""]|
+------+---------+----+

In [11]:
df.withColumn("Matricule", F.expr("""filter(Matricule, x -> x!= '""')"""))\
  .withColumn("Name", F.expr("""filter(Name, x -> x!= '""')""")).show()

+------+---------+----+
Number|Matricule|Name|
+------+---------+----+
 AA| []| [7]|
 AA| [9]| []|
 AA| []| [2]|
 AA| [2]| []|
+------+---------+----+

In [12]:
df.withColumn("Matricule", F.array_remove("Matricule", '""'))\
  .withColumn("Name", F.array_remove("Name", '""')).show()

+------+---------+----+
Number|Matricule|Name|
+------+---------+----+
 AA| []| [7]|
 AA| [9]| []|
 AA| []| [2]|
 AA| [2]| []|
+------+---------+----+

In [13]:
from pyspark.sql import functions as F
from pyspark.sql.types import *

In [14]:
list=[['2018-12-15 08:53:20','2018-12-15 08:57:57'], 
['2018-12-15 08:03:08','2018-12-15 08:07:30'],
['2018-12-15 08:28:34','2018-12-15 08:33:31'],
['2018-12-15 08:37:53','2018-12-15 08:43:47'],
['2018-12-15 08:51:02','2018-12-15 08:55:54'],
['2018-12-15 08:03:47','2018-12-15 08:03:50'],
['2018-12-15 08:45:21','2018-12-15 08:57:08'],
['2018-12-15 08:04:47','2018-12-15 08:29:05'],
['2018-12-15 08:01:22','2018-12-15 08:12:15']]
df=spark.createDataFrame(list, ['pickup','dropoff'])
df.show()
df.createOrReplaceTempView('yolo')

+-------------------+-------------------+
 pickup| dropoff|
+-------------------+-------------------+
2018-12-15 08:53:20|2018-12-15 08:57:57|
2018-12-15 08:03:08|2018-12-15 08:07:30|
2018-12-15 08:28:34|2018-12-15 08:33:31|
2018-12-15 08:37:53|2018-12-15 08:43:47|
2018-12-15 08:51:02|2018-12-15 08:55:54|
2018-12-15 08:03:47|2018-12-15 08:03:50|
2018-12-15 08:45:21|2018-12-15 08:57:08|
2018-12-15 08:04:47|2018-12-15 08:29:05|
2018-12-15 08:01:22|2018-12-15 08:12:15|
+-------------------+-------------------+

In [15]:
df.withColumn("diff", ((F.to_timestamp("dropoff")).cast("long"))-((F.to_timestamp("dropoff")).cast("long"))/60).show()

+-------------------+-------------------+--------------------+
 pickup| dropoff| diff|
+-------------------+-------------------+--------------------+
2018-12-15 08:53:20|2018-12-15 08:57:57| 1.51911653905E9|
2018-12-15 08:03:08|2018-12-15 08:07:30| 1.5191135625E9|
2018-12-15 08:28:34|2018-12-15 08:33:31|1.5191150974833333E9|
2018-12-15 08:37:53|2018-12-15 08:43:47|1.5191157032166667E9|
2018-12-15 08:51:02|2018-12-15 08:55:54| 1.5191164181E9|
2018-12-15 08:03:47|2018-12-15 08:03:50|1.5191133461666667E9|
2018-12-15 08:45:21|2018-12-15 08:57:08|1.5191164908666666E9|
2018-12-15 08:04:47|2018-12-15 08:29:05|1.5191148359166667E9|
2018-12-15 08:01:22|2018-12-15 08:12:15| 1.51911384275E9|
+-------------------+-------------------+--------------------+

In [16]:
df.withColumn("pick", F.to_timestamp("pickup").cast("long")).withColumn("drop", F.to_timestamp("dropoff").cast("long"))\
.withColumn("minutes", F.round((F.col("drop")-F.col("pick"))/60,3)).show()

+-------------------+-------------------+----------+----------+-------+
 pickup| dropoff| pick| drop|minutes|
+-------------------+-------------------+----------+----------+-------+
2018-12-15 08:53:20|2018-12-15 08:57:57|1544864000|1544864277| 4.617|
2018-12-15 08:03:08|2018-12-15 08:07:30|1544860988|1544861250| 4.367|
2018-12-15 08:28:34|2018-12-15 08:33:31|1544862514|1544862811| 4.95|
2018-12-15 08:37:53|2018-12-15 08:43:47|1544863073|1544863427| 5.9|
2018-12-15 08:51:02|2018-12-15 08:55:54|1544863862|1544864154| 4.867|
2018-12-15 08:03:47|2018-12-15 08:03:50|1544861027|1544861030| 0.05|
2018-12-15 08:45:21|2018-12-15 08:57:08|1544863521|1544864228| 11.783|
2018-12-15 08:04:47|2018-12-15 08:29:05|1544861087|1544862545| 24.3|
2018-12-15 08:01:22|2018-12-15 08:12:15|1544860882|1544861535| 10.883|
+-------------------+-------------------+----------+----------+-------+

In [17]:
spark.sql("""select pickup, dropoff, (unix_timestamp(dropoff)-unix_timestamp(pickup))/(60) as diff from yolo""").show()

+-------------------+-------------------+------------------+
 pickup| dropoff| diff|
+-------------------+-------------------+------------------+
2018-12-15 08:53:20|2018-12-15 08:57:57| 4.616666666666666|
2018-12-15 08:03:08|2018-12-15 08:07:30| 4.366666666666666|
2018-12-15 08:28:34|2018-12-15 08:33:31| 4.95|
2018-12-15 08:37:53|2018-12-15 08:43:47| 5.9|
2018-12-15 08:51:02|2018-12-15 08:55:54| 4.866666666666666|
2018-12-15 08:03:47|2018-12-15 08:03:50| 0.05|
2018-12-15 08:45:21|2018-12-15 08:57:08|11.783333333333333|
2018-12-15 08:04:47|2018-12-15 08:29:05| 24.3|
2018-12-15 08:01:22|2018-12-15 08:12:15|10.883333333333333|
+-------------------+-------------------+------------------+

In [18]:
spark.sql("""select pickup, dropoff, ((bigint(to_timestamp(dropoff)))-(bigint(to_timestamp(pickup))))/(60) as diff from yolo""").show()

+-------------------+-------------------+------------------+
 pickup| dropoff| diff|
+-------------------+-------------------+------------------+
2018-12-15 08:53:20|2018-12-15 08:57:57| 4.616666666666666|
2018-12-15 08:03:08|2018-12-15 08:07:30| 4.366666666666666|
2018-12-15 08:28:34|2018-12-15 08:33:31| 4.95|
2018-12-15 08:37:53|2018-12-15 08:43:47| 5.9|
2018-12-15 08:51:02|2018-12-15 08:55:54| 4.866666666666666|
2018-12-15 08:03:47|2018-12-15 08:03:50| 0.05|
2018-12-15 08:45:21|2018-12-15 08:57:08|11.783333333333333|
2018-12-15 08:04:47|2018-12-15 08:29:05| 24.3|
2018-12-15 08:01:22|2018-12-15 08:12:15|10.883333333333333|
+-------------------+-------------------+------------------+

In [19]:
spark.sqlContext.sql("Select to_timestamp(tpep_pickup_datetime,'MM/dd/yyyy hh:mm:ss') as pickup,to_timestamp(tpep_dropoff_datetime,'MM/dd/yyyy hh:mm:ss') as dropoff, datediff(to_timestamp(tpep_pickup_datetime,'MM/dd/yyyy hh:mm:ss'),to_timestamp(tpep_dropoff_datetime,'MM/dd/yyyy hh:mm:ss')) as diff from taxisub ").show()

In [20]:
from pyspark.sql import functions as F
list=(['cult, horror, got'],
     ['violence'],
     ['they dream of some'],
     ['imdb'],
     ['inspiring, romantic'])
df=spark.createDataFrame(list,['tags'])
df.show()

+-------------------+
 tags|
+-------------------+
 cult, horror, got|
 violence|
 they dream of some|
 imdb|
inspiring, romantic|
+-------------------+

In [21]:
list=['murder', 'violence', 'flashback', 'cult','inspiring', 'revenge', 'romantic', 'comedy', 'psychedelic', 'suspenseful', 'train']
df.withColumn("words", F.array([F.lit(string) for string in list])).withColumn("tags2", F.split(F.regexp_replace("tags", '\,',""),' ')).withColumn("toptags", F.array_intersect("tags2","words")[0]).drop("words","tags2").show()

+-------------------+---------+
 tags| toptags|
+-------------------+---------+
 cult, horror, got| cult|
 violence| violence|
 they dream of some| null|
 imdb| null|
inspiring, romantic|inspiring|
+-------------------+---------+

In [22]:
from pyspark.sql.window import Window
from pyspark.sql import functions as F

list=([1,5,4],
    [1,5,None],
    [1,5,4],
    [1,5,4],
    [2,5,1],
    [2,5,2],
    [2,5,None],
    [2,5,None])
df=spark.createDataFrame(list,['I_id','p_id','xyz'])
df.show()

+----+----+----+
I_id|p_id| xyz|
+----+----+----+
 1| 5| 4|
 1| 5|null|
 1| 5| 4|
 1| 5| 4|
 2| 5| 1|
 2| 5| 2|
 2| 5|null|
 2| 5|null|
+----+----+----+

In [23]:
w=Window().partitionBy("I_id")
df.withColumn("mean",F.mean(F.when(F.col("xyz").isNotNull(),F.col("xyz"))).over(w))\
.withColumn("xyz", F.when(F.col("xyz").isNull(),F.col("mean")).otherwise(F.col("xyz")))\
.drop("mean").show()

+----+----+---+
I_id|p_id|xyz|
+----+----+---+
 1| 5|4.0|
 1| 5|4.0|
 1| 5|4.0|
 1| 5|4.0|
 2| 5|1.0|
 2| 5|2.0|
 2| 5|1.5|
 2| 5|1.5|
+----+----+---+

In [24]:
w=Window().partitionBy("I_id","p_id")
df.withColumn("mean",F.mean("xyz").over(w))\
.withColumn("xyz", F.when(F.col("xyz").isNull(),F.col("mean")).otherwise(F.col("xyz"))).show()

+----+----+---+----+
I_id|p_id|xyz|mean|
+----+----+---+----+
 2| 5|1.0| 1.5|
 2| 5|2.0| 1.5|
 2| 5|1.5| 1.5|
 2| 5|1.5| 1.5|
 1| 5|4.0| 4.0|
 1| 5|4.0| 4.0|
 1| 5|4.0| 4.0|
 1| 5|4.0| 4.0|
+----+----+---+----+